# Generate events

This notebook shows how to generate events via Madgraph5 APIs in HML.

## Introduction to Madgraph5
[Madgraph5](https://launchpad.net/mg5amcnlo) is a framework providing all the necessary elements for studies of collider phenomenology.
It is able to generate events from a given model in UFO format, connect to a parton shower tool like
Pythia8, and perform detector simulation via tools like Delphes. It is a powerful and popular tool
in the high energy physics community.

Users typically interact with it via a command line interface (CLI). The main commands in Madgraph5
workflow are: `generate`, `output`, `launch`. `generate` is used to specify the process
of interest that allowed in the given model. `output` is used to specify where the output
files are stored. `launch` launches the generation. Then there comes two prompts: the
first one provides options of tools to use like parton shower and detector simulation, and the second
one provides related parameter cards allowing users to adjust. After the second enter or `done`, the
generation starts.

After a few tries among different parameters, a run script in plain text could be determined by users
to automate every step of Madgraph5. It's a good practice to keep the run script for consistency and
reproducibility. After reading this script, others could understand what you have done and easily
reproduce the results.

## Madgraph5 APIs in HML
While it's convenient to use the CLI, it contains a lot of jumps between the command line and the file
browser. To make it easier to acess the output `.root` file and cross section information, HML provides
`Madgraph5` class to integrate core functions of Madgraph5.

To use it, let's first import the `Madgraph5` class from `hml.generators` module.

In [1]:
from hml.generators import Madgraph5

Welcome to JupyROOT 6.24/02


As you can see, the `Madgraph5` class actually use `pyROOT` as the backend to operate on the `.root`
file. Move on, we'll see how to use it.

In [2]:
signal_generator = Madgraph5(
    processes="p p > z j, z > j j",
    output_dir="./data/pp2zj",
    shower="Pythia8",
    detector="Delphes",
    settings={
        "nevents": 10000,
        "iseed": 42,
        "htjmin": 400,
    },
)
background_generator = Madgraph5(
    processes="p p > j j j / z",
    output_dir="./data/qcd",
    shower="Pythia8",
    detector="Delphes",
    settings={
        "nevents": 10000,
        "iseed": 42,
        "htjmin": 400,
    },
)

We assume that we want to classify Z jets from ordinary QCD background, so two generators are defined.

The parameters of `Madgraph5` class are almost the same as the CLI. There are only name changes to
make it more clear and easy to understand. For example, `define` and `set` commands in CLI are
`definitions` and `settings` here.

After setting up generators, we can launch them to start generating events.

In [3]:
signal_generator.launch()
background_generator.launch()

Generating events...
Running Pythia8...
Running Delphes...
Generating events...
Running Pythia8...
Running Delphes...


Note that the `launch` method of `Madgraph5` class is different from the CLI. It doesn't launch the
prompts for tools and parameter ajustment which are now parts of `Madgraph5` parameters. Instead, it
will directly begin the generation which is more convenient for automation.

After the generation, let's check what we have got.

In [4]:
sig_run = signal_generator.runs[0]
bkg_run = background_generator.runs[0]
print(f"cross section (pb): {sig_run.cross_section}")
print(f"cross section (pb): {bkg_run.cross_section}")
print(f"number of events: {sig_run.events.GetEntries()}")
print(f"number of events: {bkg_run.events.GetEntries()}")

cross section (pb): 30.950499999999998
cross section (pb): 171411.19611000002
number of events: 10000
number of events: 10000


As each launch is corresponding to a run directory in Madgraph5, `.runs` is a intuitively way to get
all runs. Accessing cross setion is also simple via `.cross_section`. To access branches in the `.root`
file, use `.events` to open the file and operate it with `pyROOT` APIs.